In [ ]:
import pandas as pd 
import numpy as np
from sklearn.feature_selection import SelectKBest,f_classif,chi2,f_regression

In [ ]:
data = pd.read_csv("train_2019.csv")

### Implementing One Hot Encoding for Selected Features

In [ ]:
data["AIRLINE_ID"] = pd.get_dummies(data["AIRLINE_ID"])
data["DEST_AIRPORT_ID"] = pd.get_dummies(data['DEST_AIRPORT_ID'])
data['ORIGIN_AIRPORT_ID'] = pd.get_dummies(data['ORIGIN_AIRPORT_ID'])

### Handling Missing Values

In [ ]:
data = data.fillna(0)

### Time Covertion

In [ ]:
def convert_time(time):
    hour = np.floor(time/100)
    minutes = hour*60 + time - hour*100
    return minutes

In [ ]:
arr_time = data["CRS_ARR_TIME"].to_numpy().reshape(-1, 1)
dep_time = data["CRS_DEP_TIME"].to_numpy().reshape(-1, 1)
arr_time_min = convert_time(arr_time)
dep_time_min = convert_time(dep_time)
time_min_data = pd.DataFrame(np.hstack([arr_time_min, dep_time_min]), columns=['CRS_ARR_TIME_MIN', 'CRS_DEP_TIME_MIN'])

data = data.drop(["CRS_ARR_TIME", "CRS_DEP_TIME"], axis=1)
data = data.join(time_min_data)

In [ ]:
data.shape

(2216748, 37)

### Extract Target Value from Dataset

In [ ]:
target = data[["CANCELLED", "DEP_DELAY_NEW", "ARR_DELAY_NEW"]]
y = target["ARR_DELAY_NEW"].to_numpy() >0 
y = y + [0]*y.shape[0]

In [ ]:
X = data.drop(columns=["ARR_TIME","ARR_DELAY_NEW","CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY",'LATE_AIRCRAFT_DELAY',"DEP_DELAY_NEW","DEP_TIME","CRS_DEP_TIME_MIN","LONGITUDE"])
X = X._get_numeric_data()
X.columns

Index(['AIRLINE_ID', 'ORIGIN_AIRPORT_ID', 'LATITUDE', 'MONTH', 'DAY_OF_MONTH',
       'AIRPORT_FLIGHTS_MONTH', 'SEGMENT_NUMBER', 'MANUFACTURE_YEAR',
       'PLANE_AGE', 'AWND', 'PRCP', 'SNOW', 'TMAX', 'TAVG', 'DEST_AIRPORT_ID',
       'CANCELLED', 'DISTANCE', 'CRS_ARR_TIME_MIN'],
      dtype='object')

### Feature Selection - SelectKBest (for classification )

In [ ]:
from numpy import array
def select_K_Best(X,y,method,num_features):
    if method == "clf":
        select = SelectKBest(f_classif,k=num_features)
        z = select.fit_transform(X,y)
        filter = select.get_support(1)
        X_new = X[X.columns[filter]]
        selected_fs = X_new.columns
        return selected_fs,z
    else:
        select = SelectKBest(f_regression,k=num_features)
        z = select.fit_transform(X,y)
        filter = select.get_support(1)
        X_new = X[X.columns[filter]]
        selected_fs = X_new.columns
        return selected_fs,z


In [ ]:
FEATURE_NUM = 10
features_clf,z_clf = select_K_Best(X,y,"clf",FEATURE_NUM)
features_reg,z_reg = select_K_Best(X,y,"reg",FEATURE_NUM)

/Users/simon/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [15] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
/Users/simon/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/simon/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:301: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/Users/simon/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [ ]:
print("Selected Top",FEATURE_NUM," Features for Classification :",list(features_clf))

Selected Top 10  Features for Classification : ['ORIGIN_AIRPORT_ID', 'MONTH', 'AIRPORT_FLIGHTS_MONTH', 'SEGMENT_NUMBER', 'AWND', 'PRCP', 'SNOW', 'TAVG', 'DEST_AIRPORT_ID', 'CRS_ARR_TIME_MIN']


In [ ]:
print("Selected Top",FEATURE_NUM," Features for Regression :",list(features_reg))

Selected Top 10  Features for Regression : ['ORIGIN_AIRPORT_ID', 'MONTH', 'AIRPORT_FLIGHTS_MONTH', 'SEGMENT_NUMBER', 'AWND', 'PRCP', 'SNOW', 'TAVG', 'DEST_AIRPORT_ID', 'CRS_ARR_TIME_MIN']
